# Introduction to Spark on JupyterNotebook

In [1]:
# PySpark is the main library for Spark
import pyspark 
# SparkContext is the entry point for Spark functionality
from pyspark import SparkContext 
# SparkSession is the entry point for DataFrame and SQL functionality
from pyspark.sql import SparkSession 

# SQLContext is the entry point for SparkSQL functionality, providing a way to interact with structured data using SQL queries, as well as integrating with various data sources and data formats. 
#It is required to create Spark DataFrames, which are the primary data structure for working with structured data in PySpark.
from pyspark import SQLContext

In [2]:
# Provides a way of using operating system dependent functionality
import os 
# Delta is a storage layer for data lakes
from delta.tables import * 
# DeltaTable is the main class for Delta tables
from delta.tables import DeltaTable 
# Provides cryptographic hashing functions
import hashlib 
 # Provides classes for working with dates and times
import datetime
# Provides functions for working with URLs
import urllib.request 
# Provides functions for working with JSON data
import json 
 # Import timedelta and date classes from datetime module
from datetime import timedelta, date
# Provides functions for working with iterables
from itertools import islice 
# Provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter.
import sys 

ModuleNotFoundError: No module named 'delta'